In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import itertools

In [3]:
import requests
import string
import json
from ratelimit import limits, sleep_and_retry

In [4]:
from factor_model.code.passwords import alphavantage_api

In [5]:
@sleep_and_retry
@limits(calls=74, period=60)
def alphavantage_request(**kwargs):
    
    payload = ''.join([f'{key}={value}&' for key, value in kwargs.items()])
    url = f'https://www.alphavantage.co/query?{payload}&apikey={alphavantage_api}'
    r = requests.get(url)
    return r.json()    

def ticker_search(keyword):
    
    return alphavantage_request(
        function='SYMBOL_SEARCH',
        keywords=keyword
    )

In [23]:
# Load previous datastore
try:
    ticker_datastore = pd.read_csv('data/ticker_list.csv', index_col=0).transpose().to_dict()
except:
    ticker_datastore = {}
    
# Fix stored search list functionality    
def fix_searchList(store_entry):
    if type(store_entry['searchList'] == str):
        store_entry['searchList'] = json.loads(store_entry['searchList'].replace("'",'"'))
    return store_entry
ticker_datastore = {key:fix_searchList(value) for key, value in ticker_datastore.items()}

In [38]:
# Generate list of tickers to check
alphabet = string.ascii_uppercase
candidate_ticker_list = [f'{x}{y}{z}' for x in alphabet for y in alphabet for z in alphabet]
candidate_ticker_list += [f'{x}{y}' for x in alphabet for y in alphabet]
candidate_ticker_list += [x for x in alphabet]
#candidate_ticker_list += [f'{x}{y}{z}{a}' for x in alphabet for y in alphabet 
#                          for z in alphabet for a in alphabet]

# Remove already-tested symbols from the list to download
searched_list = set(itertools.chain.from_iterable(
    [value['searchList'] for key, value in ticker_datastore.items()]))
symbol_list = list(ticker_datastore.keys())
candidate_ticker_list = [x for x in candidate_ticker_list 
                         if x not in symbol_list and x not in searched_list]

# Assess list size
print(f'Hours to run loop: {len(candidate_ticker_list) / 74 / 60}')

Hours to run loop: 4.102927927927928


In [16]:
def get_matches(candidate_ticker):
    # Get ticker list    
    symbol_data_list = []
    for raw_match in ticker_search(candidate_ticker)['bestMatches']:
        symbol_data = {key.split(' ')[-1]:value for key, value in raw_match.items()}
        del symbol_data['matchScore']
        symbol_data_list.append(symbol_data)
    return symbol_data_list
    
def store_matches(symbol_data_list, lock, counter):
    # Store ticker list
    with lock:
        # Append symbol data
        global ticker_datastore
        for symbol_data in symbol_data_list:
            symbol = symbol_data['symbol']
            if symbol in ticker_datastore.keys():
                ticker_datastore[symbol]['searchList'].append(candidate_ticker)
            else:
                symbol_data['searchList'] = [candidate_ticker]
                ticker_datastore[symbol] = symbol_data
        
        # Increment save counter and store if appropriate
        counter.value+=1
        if counter.value % 750 == 0:
            pd.DataFrame(ticker_datastore).transpose().to_csv('data/ticker_list.csv')

In [17]:
import multiprocessing as mp
i = 0
lock = mp.Lock()
from ctypes import c_int
counter = mp.Value(c_int, 0)

def iterate(candidate_ticker):
    if candidate_ticker not in ticker_datastore.keys():
        try:
            symbol_data_list = get_matches(candidate_ticker)
            store_matches(symbol_data_list, lock, counter)
        except Exception as e:
            print(f'Issue with {candidate_ticker}: {e}')

In [ ]:
for candidate_ticker in tqdm(candidate_ticker_list[150:]):
    iterate(candidate_ticker)

 22%|██▏       | 3935/18067 [52:51<1:30:22,  2.61it/s] 

In [31]:
ticker_df = pd.DataFrame(ticker_datastore).transpose()

In [36]:
ticker_df[ticker_df.searchList.apply(lambda x: 'AES' in x)]

,symbol,name,type,region,marketOpen,marketClose,timezone,currency,searchList
AES,AES,AES Corp,Equity,United States,09:30,16:00,UTC-04,USD,[AES]
AESI,AESI,Atlas Energy Solutions Inc,Equity,United States,09:30,16:00,UTC-04,USD,[AES]
AESO,AESO,Atlantic Energy Solutions Inc,Equity,United States,09:30,16:00,UTC-04,USD,[AES]
AESR,AESR,Anfield U.S. Equity Sector Rotation ETF,ETF,United States,09:30,16:00,UTC-04,USD,[AES]
AES.FRK,AES.FRK,The AES Corporation,Equity,Frankfurt,08:00,20:00,UTC+02,EUR,[AES]
AESTY,AESTY,Aes Tiete Energia S.A.,Equity,United States,09:30,16:00,UTC-04,USD,[AES]
AESB3.SAO,AESB3.SAO,AES Brasil Energia S.A,Equity,Brazil/Sao Paolo,10:00,17:30,UTC-03,BRL,[AES]
0H6G.LON,0H6G.LON,AES Corp.,Equity,United Kingdom,08:00,16:30,UTC+01,USD,[AES]
IPWLG,IPWLG,AES Indiana,Equity,United States,09:30,16:00,UTC-04,USD,[AES]
RBED11.SAO,RBED11.SAO,Aesapar Fundo de Investimento Imobiliário - FII,Mutual Fund,Brazil/Sao Paolo,10:00,17:30,UTC-03,BRL,[AES]
